In [ ]:
# Install ES bagi yang belum install
!python -m pip install elasticsearch

In [17]:
import pandas as pd
import warnings

from elasticsearch import Elasticsearch
from elasticsearch import helpers

warnings.filterwarnings('ignore')

In [ ]:
# Connect ke server Elasticsearch di localhost port 9200
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])
es.info()

{'name': 'Ruanggurus-MacBook-Air-2.local',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'FhIc_VuISt21nwpqpH-xxw',
 'version': {'number': '7.17.21',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd38e4b028f4a9784bb74de339ac1b877e2dbea6f',
  'build_date': '2024-04-26T04:36:26.745220156Z',
  'build_snapshot': False,
  'lucene_version': '8.11.3',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [ ]:
df = pd.read_csv('flipkart_sample.csv')
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,a495f1924739a491decf660e7e3d8070,2016-04-18 03:49:23 +0000,http://www.flipkart.com/shopoj-multicolor-pape...,SHOPOJ Multicolor Paper Sky Lantern,"[""SHOPOJ Multicolor Paper Sky Lantern (80 cm X...",LTNEHQSPYVUJVBT5,899.0,515.0,"[""http://img6a.flixcart.com/image/lantern/b/t/...",False,Key Features of SHOPOJ Multicolor Paper Sky La...,No rating available,No rating available,SHOPOJ,"{""product_specification""=>[{""key""=>""Brand"", ""v..."
1,7bb2c1267e87cbeb9b2cf0720615cc49,2016-01-07 05:50:25 +0000,http://www.flipkart.com/vgreen-plant-container...,VGreen Plant Container Set,"[""Tools & Hardware >> Tools >> Gardening Tools...",PCSE9R7E3AFG7F9H,400.0,340.0,"[""http://img5a.flixcart.com/image/plant-contai...",False,Buy VGreen Plant Container Set for Rs.340 onli...,No rating available,No rating available,VGreen,"{""product_specification""=>[{""key""=>""Shape"", ""v..."
2,b9cb9c703567e1b35f5d2cb6b248598b,2016-02-24 02:50:54 +0000,http://www.flipkart.com/royal-canin-maxi-start...,Royal Canin Maxi Starter 1kg Vegetable Dog Food,"[""Pet Supplies >> Food & Health Supplies >> Fo...",PFDEG3AHQ4SBVDQB,660.0,650.0,"[""http://img5a.flixcart.com/image/pet-food/d/q...",False,Specifications of Royal Canin Maxi Starter 1kg...,No rating available,No rating available,Royal Canin,"{""product_specification""=>[{""key""=>""Pet Type"",..."
3,6c5d4e90fdd095f1c572440e826e89aa,2016-01-07 06:04:02 +0000,http://www.flipkart.com/showtime-women-s-full-...,"ShowTime Women's Full Coverage, T-Shirt Bra","[""Clothing >> Women's Clothing >> Lingerie, Sl...",BRAEE3HEHFEYSXRZ,299.0,135.0,"[""http://img5a.flixcart.com/image/bra/x/r/z/ju...",False,"ShowTime Women's Full Coverage, T-Shirt Bra\n ...",No rating available,No rating available,Beige,"{""product_specification""=>[{""key""=>""Brand Colo..."
4,66ec648a06218a2a9c2330c379837642,2015-12-01 06:13:00 +0000,http://www.flipkart.com/hi-look-alloy-necklace...,Hi Look Alloy Necklace,"[""Jewellery >> Necklaces & Chains >> Necklaces""]",NKCEFMKX9GH6GSTS,695.0,295.0,"[""http://img6a.flixcart.com/image/necklace-cha...",False,Hi Look Alloy Necklace - Buy Hi Look Alloy Nec...,No rating available,No rating available,Hi Look,"{""product_specification""=>[{""key""=>""Collection..."


In [3]:
# Pilih beberapa kolom yang akan dipakai

columns_selected = ['uniq_id', 'product_name', 'retail_price', 'discounted_price',
                    'is_FK_Advantage_product', 'description', 'product_rating', 'overall_rating', 'brand']

df_selected = df[columns_selected]
df_selected.head()

,uniq_id,product_name,retail_price,discounted_price,is_FK_Advantage_product,description,product_rating,overall_rating,brand
0,a495f1924739a491decf660e7e3d8070,SHOPOJ Multicolor Paper Sky Lantern,899.0,515.0,False,Key Features of SHOPOJ Multicolor Paper Sky La...,No rating available,No rating available,SHOPOJ
1,7bb2c1267e87cbeb9b2cf0720615cc49,VGreen Plant Container Set,400.0,340.0,False,Buy VGreen Plant Container Set for Rs.340 onli...,No rating available,No rating available,VGreen
2,b9cb9c703567e1b35f5d2cb6b248598b,Royal Canin Maxi Starter 1kg Vegetable Dog Food,660.0,650.0,False,Specifications of Royal Canin Maxi Starter 1kg...,No rating available,No rating available,Royal Canin
3,6c5d4e90fdd095f1c572440e826e89aa,"ShowTime Women's Full Coverage, T-Shirt Bra",299.0,135.0,False,"ShowTime Women's Full Coverage, T-Shirt Bra\n ...",No rating available,No rating available,Beige
4,66ec648a06218a2a9c2330c379837642,Hi Look Alloy Necklace,695.0,295.0,False,Hi Look Alloy Necklace - Buy Hi Look Alloy Nec...,No rating available,No rating available,Hi Look


In [ ]:
# Buat index baru di Elasticsearch
response = es.indices.create(index='flipkart_products', ignore=400)
response

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'flipkart_products'}

In [12]:
# Get semua index yang ada di instance Elasticsearch
response = es.cat.indices()
print(response)

green  open .geoip_databases  tondDws9Q9OWZge_OATujg 1 0   37 0  35.2mb  35.2mb
yellow open orders            5Hjs5GGTSqmAk34WuHywzw 1 1    1 0   5.8kb   5.8kb
yellow open customers         7XP5r0IGSJ-OfKSE0bWc7A 1 1 2000 0 557.1kb 557.1kb
yellow open flipkart_products j0Vqxv6sRPKfAkAtHJCI8A 1 1    0 0  86.4kb  86.4kb
yellow open customers4        -DRIPVObTsOM6_ONwW2AMA 1 1    3 1   8.5kb   8.5kb
yellow open task_details      akF9uPhWTyCuNwjQAPTf5Q 1 1    1 0    25kb    25kb
yellow open products          FkW_WXNCQZC_J_3AJtBo7g 1 1    5 1  13.3kb  13.3kb



In [14]:
response = es.indices.put_settings(
        index='flipkart_products',
        body={
            "index": {
                "blocks": {
                    "read_only_allow_delete": False
                }
            }
        }
    )

In [ ]:
df_selected.dropna(inplace=True)

In [ ]:
# Insert data ke index yang baru dibuat menggunakan Elasticsearch (bulk)
helpers.bulk(es, df_selected.to_dict(orient='records'), index='flipkart_products', doc_type='_doc')

In [ ]:
# Melakukan query untuk mencari data yang ada di index flipkart_products
es.search(index='flipkart_products', body={'query': {'match_all': {}}})